In [1]:
# Exploration Notebook to 
#  - find and correct the idiosyncrasies of the Interties lookup table as it came from Neil
#  - test chunks of code before they were formalized in interties2grids.py
#  - test the final output to ensure it is doing expected things
# run with "uv run --with jupyter jupyter lab" with the venv activated

from pathlib import Path

import numpy as np
import pandas as pd

infile = "../../data/raw/dowl/lookup_interties_2023-11-08.csv"
outfile = "lookup_grids_2025-03-31.csv"


In [2]:
# read
df = pd.read_csv(infile)

# some columns we don't need
df.drop(columns=[
    "month_of_intertie",   # mostly unknown
    "source", # awkward. maybe add later
    "aea_energy_region",  # in the crosswalk or maybe we aren't using
], inplace=True)
# Railbelt and Copper Valley have blank placeholders. Can't deal with that
df.dropna(subset=['communities_intertied'], inplace=True)

# Standardize the grid names - some have spaces or punctuation
print(len(df['intertie_unique_id_name'].unique()))
df['intertie_unique_id_name'] = df['intertie_unique_id_name'].str.replace(' ', '').str.replace('.', '').str.replace('-', '').str.replace('\'', '')
df['intertie_unique_id_name'] = df['intertie_unique_id_name'].str.replace('_grid', '').str.replace('_', '')
df['intertie_unique_id_name'] = df['intertie_unique_id_name'] + '_grid'
print(len(df['intertie_unique_id_name'].unique()))

# reformat the id
df['intertie_id'] = df['intertie_id'].str.replace('_', '-')
df[['grid_id', 'connection_year']] = df['intertie_id'].str.split('-', expand=True)
df['ak_grid_id'] = 'AK-GR-' + df['grid_id']
df['grid_id'] = df['grid_id'].astype(int)
df['connection_year'] = df['connection_year'].astype(int)

# to separate communities into rows, first standardize delimiters (including dash in "Wrangell-Petersburg"
df['communities_intertied'] = df['communities_intertied'].str.replace(';', ',').str.replace('-', ',')
df['communities_intertied'] = df['communities_intertied'].map(lambda x: [y.strip() for y in x.split(',')])

df

186
185


,intertie_id,intertie_unique_id_name,current_id,communities_intertied,year_of_intertie,grid_id,connection_year,ak_grid_id
0,001-0000,Akhiok_grid,True,[Akhiok],NaN,1,0,AK-GR-001
1,002-0000,Akiachak_grid,True,[Akiachak],NaN,2,0,AK-GR-002
2,003-0000,Akiak_grid,True,[Akiak],NaN,3,0,AK-GR-003
3,004-0000,Akutan_grid,True,[Akutan],NaN,4,0,AK-GR-004
4,005-0000,Allakaket_grid,True,"[Allakaket, Alatna]",NaN,5,0,AK-GR-005
...,...,...,...,...,...,...,...,...
200,224-0000,Deadhorse_grid,True,[Deadhorse],NaN,224,0,AK-GR-224
201,225-0000,Metlakatla_grid,True,[Metlakatla],NaN,225,0,AK-GR-225
202,226-0000,Paxson_grid,True,[Paxson],NaN,226,0,AK-GR-226
203,227-0000,Ketchikan_grid,False,[Ketchikan],NaN,227,0,AK-GR-227


In [3]:
# sometimes grids start with many communities. Prince_of_Wales_Is_grid added 2 communities at a time
# so cann't assume building up one by one
number_changed = {}
for grid_name in df['intertie_unique_id_name'].unique():
    list_lens = []
    grid = df.loc[grid_name == df['intertie_unique_id_name']]
    for index, community_list in grid['communities_intertied'].items():
        if grid_name == 'Prince_of_Wales_Is_grid':
            print(grid_name, community_list)
        #print(grid_name, len(value))
        list_lens.append(len(community_list))
    if list_lens[0] != 1 or len(list_lens) > 1:
        print(grid_name, list_lens)
    if len(list_lens) > 1:
        number_changed[grid_name] = list_lens

Allakaket_grid [2]
Bettles_grid [2]
Eagle_grid [2]
Northway_grid [3]
Cordova_grid [2]
Tok_grid [4, 5]
PrinceofWalesIs_grid [2, 4, 5, 6, 7, 8]
Slana_grid [1, 2, 3]
Haines_grid [4]
UpperLynnCanal_grid [5]
Bethel_grid [2, 3]
Emmonak_grid [1, 2]
UpperKalskag_grid [1, 2]
Kasigluk_grid [1, 2]
NewStuyahok_grid [1, 2]
SaintMarys_grid [3, 4]
Stebbins_grid [1, 2]
Togiak_grid [1, 2]
ToksookBay_grid [1, 2, 3]
Newhalen_grid [3]
Shungnak_grid [2]
Naknek_grid [3]
Dillingham_grid [2]
SEAPA_grid [3]
Juneau_grid [2]
Kodiak_grid [5]
WrangellPetersburg_grid [2]


In [4]:
# list of all communities mentioned
all_communities = []
for index, community_list in df['communities_intertied'].items():
    all_communities = all_communities + community_list

all_communities = list(set(all_communities))
all_communities.sort()
print(f"{len(all_communities)} communities")

# sometimes grids are not named after a community in the list (or punctuation is different)
all_grid_names = df['intertie_unique_id_name'].unique()
print(f"{len(all_grid_names)} grids")

for gn in all_grid_names:
    match = False
    gn_clean = gn.replace('_grid', '')
    for cn in all_communities:
        if cn.replace(' ', '') in gn:
            match = True
            break
        elif gn_clean in cn.replace(' ', '').replace('\'', ''):
            match = True
            break    
    if not match:
        print(gn)
        


214 communities
185 grids
UpperLynnCanal_grid
SEAPA_grid


In [5]:
# how many grids is each community mentioned in?
comcount = {}
for community in all_communities:
    count = 0
    for grid_name in df['intertie_unique_id_name'].unique():
        grid = df.loc[grid_name == df['intertie_unique_id_name']]
        for index, community_list in grid['communities_intertied'].items():
            if community in community_list:
                count = count + 1
    if count > 1:
        print(community, count)
        comcount[community] = count

Alakanuk 2
Andreafsky 2
Bethel 2
Chilkat Valley 2
Chistochina 3
Coffman Cove 3
Covenant Life 2
Craig 6
Dot Lake 2
Dot Lake Village 2
Ekwok 2
Emmonak 2
Haines 2
Hollis 5
Hydaburg 4
Kalskag 2
Kasaan 5
Kasigluk 2
Ketchikan 2
Klawock 6
Klukwan 2
Kobuk 2
Lower Kalskag 2
Mentasta Lake 2
Mountain Village 2
Napakiak 2
Naukati Bay 2
New Stuyahok 2
Nightmute 2
Nunapitchuk 2
Petersburg 2
Pitkas Point 3
Saint Mary's 2
Saint Michael 2
Slana 3
Stebbins 2
Tanacross 2
Tetlin 2
Thorne Bay 5
Togiak 2
Tok 2
Toksook Bay 2
Tununak 3
Twin Hills 2
Wrangell 2


In [6]:
# associate the community name with the year it was added to a grid
print(number_changed)
events = pd.DataFrame({})
for key, counts in number_changed.items():
    grid = df.loc[key== df['intertie_unique_id_name']]
    previous = grid['communities_intertied'].values[0]
    for index, row in grid.iloc[1:, :].iterrows():
        event = {}
        event['to_grid_name'] = key
        event['to_grid_id'] = row['ak_grid_id']
        event['year'] = row['connection_year']
        event['added'] = list( set(row['communities_intertied']) - set(previous) )
        previous = row['communities_intertied']
        events = pd.concat([events, pd.DataFrame(event)], ignore_index=True)
events.set_index('added', inplace=True)
print(events)
print(events.loc['Tetlin'])

{'Tok_grid': [4, 5], 'PrinceofWalesIs_grid': [2, 4, 5, 6, 7, 8], 'Slana_grid': [1, 2, 3], 'Bethel_grid': [2, 3], 'Emmonak_grid': [1, 2], 'UpperKalskag_grid': [1, 2], 'Kasigluk_grid': [1, 2], 'NewStuyahok_grid': [1, 2], 'SaintMarys_grid': [3, 4], 'Stebbins_grid': [1, 2], 'Togiak_grid': [1, 2], 'ToksookBay_grid': [1, 2, 3]}
                          to_grid_name to_grid_id  year
added                                                  
Tetlin                        Tok_grid  AK-GR-015  2008
Thorne Bay        PrinceofWalesIs_grid  AK-GR-023  1999
Kasaan            PrinceofWalesIs_grid  AK-GR-023  1999
Hollis            PrinceofWalesIs_grid  AK-GR-023  2004
Hydaburg          PrinceofWalesIs_grid  AK-GR-023  2006
Coffman Cove      PrinceofWalesIs_grid  AK-GR-023  2011
Naukati Bay       PrinceofWalesIs_grid  AK-GR-023  2015
Chistochina                 Slana_grid  AK-GR-026  2010
Mentasta Lake               Slana_grid  AK-GR-026  2012
Oscarville                 Bethel_grid  AK-GR-034  1988
Alak

In [7]:
# Not all added communities have their own grid to start with in the Intertie table
from_grid = []
for community, row in events.iterrows():
    alone = False
    for index2, row2 in df.iterrows():
        if [community] == row2['communities_intertied']:
            #from_grid.append(row2['intertie_unique_id_name'])
            events.loc[community, 'from_grid_name'] = row2['intertie_unique_id_name']
            events.loc[community, 'from_grid_id'] = row2['ak_grid_id']

events

,to_grid_name,to_grid_id,year,from_grid_name,from_grid_id
added,,,,,
Tetlin,Tok_grid,AK-GR-015,2008,Tetlin_grid,AK-GR-212
Thorne Bay,PrinceofWalesIs_grid,AK-GR-023,1999,NaN,NaN
Kasaan,PrinceofWalesIs_grid,AK-GR-023,1999,NaN,NaN
Hollis,PrinceofWalesIs_grid,AK-GR-023,2004,Hollis_grid,AK-GR-209
Hydaburg,PrinceofWalesIs_grid,AK-GR-023,2006,Hydaburg_grid,AK-GR-210
Coffman Cove,PrinceofWalesIs_grid,AK-GR-023,2011,CoffmanCove_grid,AK-GR-207
Naukati Bay,PrinceofWalesIs_grid,AK-GR-023,2015,NaukatiBay_grid,AK-GR-211
Chistochina,Slana_grid,AK-GR-026,2010,Chistochina_grid,AK-GR-206
Mentasta Lake,Slana_grid,AK-GR-026,2012,MentastaLake_grid,AK-GR-214


In [8]:
len(events)

20

In [9]:
## Exploration is done
# ready to build the Grids table

grids = pd.DataFrame({})
for grid_id in df['ak_grid_id'].unique():
    chunk = df.loc[grid_id == df['ak_grid_id']]
    record = chunk.loc[chunk['connection_year'] == chunk['connection_year'].min(), :]
    grids = pd.concat([grids, record], ignore_index=True)

grids = grids.explode(column='communities_intertied', ignore_index=True)
grids['termination_year'] = np.nan
print(grids.shape)

for index, row in events.iterrows():
    # add the connection info
    joining_grid = row['to_grid_id']
    year = row['year']
    record = df.loc[(df['ak_grid_id'] == joining_grid) & (df['connection_year'] == year), :]
    record.loc[:,'communities_intertied'] = index
    grids = pd.concat([grids, record], ignore_index=True)

    # alter existing record to show termination info
    leaving_grid = row['from_grid_id']
    grids.loc[grids['ak_grid_id'] == leaving_grid, 'termination_year'] = year

# assume if no termination year, then it is still operating
grids['termination_year'] = grids['termination_year'].fillna(9999)
grids['termination_year'] = grids['termination_year'].astype(int)


# clear out last vestiges
grids.drop(columns=[
    'intertie_id',
    'current_id',
    'year_of_intertie'
], inplace=True)
grids.rename(columns={'communities_intertied': 'community', 'intertie_unique_id_name': 'grid_name'}, inplace=True)

grids = grids[['community', 'grid_id', 'ak_grid_id', 'grid_name', 'connection_year', 'termination_year']]
grids.sort_values(by=['grid_id', 'connection_year'], inplace=True)

grids

(214, 9)


,community,grid_id,ak_grid_id,grid_name,connection_year,termination_year
0,Akhiok,1,AK-GR-001,Akhiok_grid,0,9999
1,Akiachak,2,AK-GR-002,Akiachak_grid,0,9999
2,Akiak,3,AK-GR-003,Akiak_grid,0,9999
3,Akutan,4,AK-GR-004,Akutan_grid,0,9999
4,Allakaket,5,AK-GR-005,Allakaket_grid,0,9999
...,...,...,...,...,...,...
209,Metlakatla,225,AK-GR-225,Metlakatla_grid,0,9999
210,Paxson,226,AK-GR-226,Paxson_grid,0,9999
211,Ketchikan,227,AK-GR-227,Ketchikan_grid,0,9999
212,Wrangell,228,AK-GR-228,WrangellPetersburg_grid,1981,9999


In [10]:
grids.loc[grids['community'] == 'Mountain Village']



,community,grid_id,ak_grid_id,grid_name,connection_year,termination_year
63,Mountain Village,64,AK-GR-064,MountainVillage_grid,0,2020
228,Mountain Village,75,AK-GR-075,SaintMarys_grid,2020,9999


In [11]:
grids.loc[grids['grid_name'] == 'SaintMarys_grid']

,community,grid_id,ak_grid_id,grid_name,connection_year,termination_year
60,Saint Mary's,75,AK-GR-075,SaintMarys_grid,1985,9999
61,Andreafsky,75,AK-GR-075,SaintMarys_grid,1985,9999
62,Pitkas Point,75,AK-GR-075,SaintMarys_grid,1985,9999
228,Mountain Village,75,AK-GR-075,SaintMarys_grid,2020,9999


In [12]:
grids.loc[grids['ak_grid_id'] == 'AK-GR-023']

,community,grid_id,ak_grid_id,grid_name,connection_year,termination_year
20,Craig,23,AK-GR-023,PrinceofWalesIs_grid,1987,9999
21,Klawock,23,AK-GR-023,PrinceofWalesIs_grid,1987,9999
215,Thorne Bay,23,AK-GR-023,PrinceofWalesIs_grid,1999,9999
216,Kasaan,23,AK-GR-023,PrinceofWalesIs_grid,1999,9999
217,Hollis,23,AK-GR-023,PrinceofWalesIs_grid,2004,9999
218,Hydaburg,23,AK-GR-023,PrinceofWalesIs_grid,2006,9999
219,Coffman Cove,23,AK-GR-023,PrinceofWalesIs_grid,2011,9999
220,Naukati Bay,23,AK-GR-023,PrinceofWalesIs_grid,2015,9999
